In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
df

In [ ]:
X = df.iloc[:,:-1].values 
y = df.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

In [ ]:
import kerastuner as kt

In [ ]:
#for optimizers
def build_model(hp):
    
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    optimizer=hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adagrad','adadelta'])
    
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    
    return model
    
    

In [ ]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,epochs=100,batch_size=32,initial_epoch=6,validation_data=(X_test,y_test))

In [ ]:
#for units in each layer
def build_model(hp):
    
    model = Sequential()
    units = hp.Int('units',min_value=8,max_value=128,step=8)
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5,
                       directory='mydir')

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,epochs=100,batch_size=32,initial_epoch=6,validation_data=(X_test,y_test))

In [ ]:
#for layers

def build_model(hp):
    
    model = Sequential()
    model.add(Dense(104,activation='relu',input_dim=8))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(104,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                  objective='val_accuracy',
                  max_trials=3,
                directory='mydir',
                project_name='num_layers')

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
#tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test))